In [1]:
#jupyter
%load_ext autoreload
%autoreload 2

#own packages
#from my_packages.comments_filter import emojies_re_pattern
#from my_packages.process_timestamps_update import plot_time_stamps
#from my_packages.download_comments_update import download_comments


#standart packages
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from langdetect import detect

#machine learning packages
from simpletransformers.classification import ClassificationModel, ClassificationArgs
# https://pypi.org/project/simpletransformers/ <- source evaluation tipps
import pandas as pd
from sys import argv
from sys import stderr
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

/opt/anaconda3/envs/my_ml_env2/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# create Annotation List
df1 = pd.read_csv("Ed-Sheeran-1000-PP.tsv", sep="\t", nrows=500)
df2 = pd.read_csv("John-Legend-1000-PP.tsv", sep="\t", nrows=500)
df3 = pd.read_csv("Kelly-Clarkson-1000-PP.tsv", sep="\t", nrows=500)
df4 = pd.read_csv("Tom-Odell-1000-PP.tsv", sep="\t", nrows=500)

df1 = df1[df1["english"] == True]
print(len(df1))
df2 = df2[df2["english"] == True]
print(len(df2))
df3 = df3[df3["english"] == True]
print(len(df3))
df4 = df4[df4["english"] == True]
print(len(df4))

df1 = df1.head(250)
df2 = df2.head(250)
df3 = df3.head(250)
df4 = df4.head(250)

df_final = pd.concat([df1, df2, df3, df4])

print(df_final)
filename = "final_annotation.tsv"
df_final.to_csv(filename, sep="\t")

In [4]:
## import *finished* annotated file
filename = "annotation_data/annotate_1000_final.tsv"
annotations = pd.read_csv(filename, sep="\t")
selected_columns = ["author", "comment", "origin", "origin_status", "category"]
annotations = annotations[selected_columns]
print(annotations.columns)

Index(['author', 'comment', 'origin', 'origin_status', 'category'], dtype='object')


In [5]:
## prepare Dataset
#delete non-category
annotations = annotations[annotations["category"] != -1]

#shuffle data and drop old index
annotations = annotations.sample(frac=1, random_state=1).reset_index(drop=True)

# create training-data <-- df
train_df = annotations.head(int(len(annotations)*0.8))
selected_columns = ["comment", "category"]
train_df = train_df[selected_columns]

# create eval-data <-- df
eval_df = annotations.tail(int(len(annotations)*0.2))
selected_columns = ["comment", "category"]
eval_df = eval_df[selected_columns]

print(len(train_df))
print(len(eval_df))

752
188


In [6]:
#Train Model and Save it

# toggle logging
#from transformers.utils import logging
#logging.set_verbosity_info()

#TOKENIZERS_PARALLELISM=True

# Optional model configuration
model_args = ClassificationArgs(
                                num_train_epochs=1, 
                                overwrite_output_dir= True
                                )

model = ClassificationModel(
                            'distilbert',
                            'distilbert-base-uncased',
                            use_cuda=False,
                            args=model_args
                            )
#--> set cuda to True, if available

# start training#
model.train_model(train_df)

#save model
#modelname = "model_p82"
#model.model.save_pretrained(modelname)
#model.tokenizer.save_pretrained(modelname)
#model.config.save_pretrained(f'{modelname}/')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier

Epochs 0/1. Running Loss:    0.3025:  69%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                               | 65/94 [01:28<00:39,  1.36s/it]


Epoch 1 of 1: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [02:09<00:00, 129.83s/it]


(94, 0.4837440553497761)

In [8]:
#evaluate model

# reload model (if needed)
model = ClassificationModel('distilbert','model_p82', use_cuda=False)

# start evaluation
print("Starting Evaluation:")
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

# print results
print("Results:")
for i in result:
    print(i, result[i])

#tp = true positives
#tn = true negatives
#fp = false positives
#fn = false negaives
#auroc = precision
#auprc = average precison
    
print("Precision:")
print((len(eval_df)-len(wrong_predictions))/len(eval_df))
    
print(len(wrong_predictions))

# print all wrong preductions (if wanted)
for n, dic in enumerate(wrong_predictions):
    print("Annotation Label:", dic.label)
    print(dic.text_a)
    print(10*'+')

Starting Evaluation:


/opt/anaconda3/envs/my_ml_env2/lib/python3.10/site-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
Running Evaluation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:05<00:00,  4.15it/s]

Results:
mcc 0.5251765486267972
tp 29
tn 126
fp 13
fn 20
auroc 0.8684480986639259
auprc 0.6599662218317867
eval_loss 0.40511993194619816
Precision:
0.824468085106383
33
Annotation Label: 1
When you are happy you enjoy the song and when you’re sad you understand the lyrics
++++++++++
Annotation Label: 0
Lyrics ....

When your legs don't work like they used to before
And I can't sweep you off of your feet
Will your mouth still remember the taste of my love
Will your eyes still smile from your cheeks
And darling I will be loving you 'til we're 70
And baby my heart could still fall as hard at 23
And I'm thinking 'bout how people fall in love in mysterious ways
Maybe just the touch of a hand
Oh me I fall in love with you every single day
And I just wanna tell you I am
So honey now
Take me into your loving arms
Kiss me under the light of a thousand stars
Place your head on my beating heart
I'm thinking out loud
Maybe we found love right where we are
When my hair's all but gone and my memory 

# Apply Model

In [4]:
# prepare Application-Dataset (input needs to be a list)
filename = "comment_downloads/data_combined_53221.tsv"
origin = "Kelly_Clarkson-Because_Of_You"
time_stamp = False


apply_df = pd.read_csv(filename, sep="\t")
apply_df = apply_df[apply_df["origin"] == origin]
apply_df = apply_df[apply_df["time_stamp"] == time_stamp]



print(apply_df)
apply_lst = apply_df['comment'].tolist()

print(len(apply_lst))

       Unnamed: 0.2 Unnamed: 0.1 Unnamed: 0            author  \
10274         10274            0          0     Bruce VanBeek   
10275         10275            3          3         nayneeeee   
10276         10276            4          4  letitia marshall   
10277         10277            7          7           soni uk   
10278         10278            8          8  Christian Havana   
...             ...          ...        ...               ...   
21017         21017        20080      20080          ScarWølf   
21018         21018        20081      20081            Ronald   
21019         21019        20082      20082         Rahaf Mrd   
21020         21020        20085      20085           Bela S.   
21021         21021        20086      20086    shrooq mujarbi   

                                                 comment  \
10274  I know a daughter of a good man . Since her mo...   
10275  In every failed relationship it was always the...   
10276                                  

In [3]:
# load model
model = ClassificationModel('distilbert','model_p82', use_cuda=False)

#apply model
predictions, raw_outputs = model.predict(apply_lst)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:04<00:00,  3.62it/s]


In [4]:
import sys
#sys.stdout = open('output.txt','a')
print(20*"-")
print(filename, origin, time_stamp)

# output and save predictions
final = []
for i, prediction in enumerate(predictions):
  comment_result = (apply_lst[i], str(prediction), str(raw_outputs[i]))
  final.append(comment_result)
df = pd.DataFrame(final, columns =['txt', 'prediction', 'probablility'])

#filename = 'output.tsv'
#df.to_csv(filename, sep='\t')
counts = df["prediction"].value_counts()
print("1 = schnulzig; 0 = nicht-schnulzig")
print(counts)
print(f"results saved in {filename}")

perc = counts.iloc[1]/(counts.iloc[0]+counts.iloc[1])
perc = '{:,.2%}'.format(perc)
print(f"{perc} der Kommentare wurden als schnulzig klassifiziert.")


--------------------
comment_downloads/data_combined_53221.tsv Kelly_Clarkson-Because_Of_You True
1 = schnulzig; 0 = nicht-schnulzig
0    89
1    24
Name: prediction, dtype: int64
results saved in comment_downloads/data_combined_53221.tsv
21.24% der Kommentare wurden als schnulzig klassifiziert.


In [3]:
# encoding=utf-8
import sys
#machine learning packages
from simpletransformers.classification import ClassificationModel, ClassificationArgs
# https://pypi.org/project/simpletransformers/ <- source evaluation tipps
import pandas as pd
from sys import argv
from sys import stderr
import os
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.INFO)


os.environ["TOKENIZERS_PARALLELISM"] = "false"


origin = "Kelly_Clarkson-Because_Of_You"
time_stamp = False



# prepare Application-Dataset (input needs to be a list)
filename = "comment_downloads/data_combined_53221.tsv"
apply_df = pd.read_csv(filename, sep="\t")
apply_df = apply_df[apply_df["origin"] == origin]
apply_df = apply_df[apply_df["time_stamp"] == time_stamp]
apply_lst = apply_df['comment'].tolist()
apply_lst = apply_lst[:5]

print(len(apply_lst))
# load model
model = ClassificationModel('distilbert','model_p82', use_cuda=False)
#apply model

com_lst = []
pred_lst = []

for i, comment in enumerate(apply_lst):
    single_lst = []
    single_lst.append(comment)
    comment = single_lst
    print(i)
    predictions, raw_outputs = model.predict(comment)
    com_lst.append(comment)
    pred_lst.append(predictions)

print(20*"-")
print(filename, origin, time_stamp)

# output and save predictions
final = []
for i, comment in enumerate(com_lst):
  comment_row = (comment, str(pred_lst[i]))
  final.append(comment_row)

df = pd.DataFrame(final, columns =['txt', 'prediction'])

print("1 = schnulzig; 0 = nicht-schnulzig")
counts = df["prediction"].value_counts()
print(counts)
try:
    perc = counts.iloc[1]/(counts.iloc[0]+counts.iloc[1])
except:
    perc = 0
perc = '{:,.2%}'.format(perc)
print(f"{perc} der Kommentare wurden als schnulzig klassifiziert.")


filename = 'output.tsv'
df.to_csv(filename, sep='\t')
print(f"results saved in {filename}")

loading configuration file model_p82/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}

loading weights file model_p82/pytorch_model.bin


5


All model checkpoint weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the model checkpoint at model_p82.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.
loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


0


  0%|                                                                                                                                                                                                | 0/1 [00:00<?, ?it/s]Process SpawnPoolWorker-105:
Process SpawnPoolWorker-109:
Process SpawnPoolWorker-103:
Process SpawnPoolWorker-108:
Process SpawnPoolWorker-107:
Process SpawnPoolWorker-104:
Process SpawnPoolWorker-101:
Process SpawnPoolWorker-106:
Process SpawnPoolWorker-110:
  0%|                                                                                                                                                                                                | 0/1 [00:05<?, ?it/s]Process SpawnPoolWorker-102:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/anaconda3/envs/my_ml_env2/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/my_ml_env2/lib/python3.10/multiprocessing/process.py

KeyboardInterrupt: 